In [3]:
import os

In [4]:
%pwd

'c:\\Users\\User\\Desktop\\MLOps Project\\End-to-End-ML-Project\\research'

In [5]:
os.chdir("../")

In [34]:
%pwd

'c:\\Users\\User\\Desktop\\MLOps Project\\End-to-End-ML-Project'

In [35]:
from dataclasses import dataclass
from pathlib import Path

@dataclass(frozen=True)
class DataIngestionConfig:
 root_dir : Path
 source_URL : str
 local_data_file: Path
 unzip_dir: Path

In [36]:
%ls

 Volume in drive C has no label.
 Volume Serial Number is 9601-F3BC

 Directory of c:\Users\User\Desktop\MLOps Project\End-to-End-ML-Project

01/28/2024  04:33 PM    <DIR>          .
01/28/2024  04:33 PM    <DIR>          ..
01/28/2024  11:59 AM             3,238 .gitignore
01/28/2024  12:55 PM                 0 app.py
01/28/2024  12:55 PM    <DIR>          config
01/28/2024  11:59 AM             1,089 LICENSE
01/28/2024  04:33 PM    <DIR>          logs
01/28/2024  04:38 PM                82 main.py
01/28/2024  06:45 PM                 9 params.yaml
01/28/2024  06:13 PM               517 README.md
01/28/2024  01:19 PM               138 requirements.txt
01/28/2024  06:31 PM    <DIR>          research
01/28/2024  06:44 PM               379 schema.yaml
01/28/2024  04:20 PM               771 setup.py
01/28/2024  01:25 PM    <DIR>          src
01/28/2024  12:55 PM             1,266 template.py
01/28/2024  12:55 PM    <DIR>          templates
              10 File(s)          7,489 bytes
   

In [42]:
from src.MLProject.constants import CONFIG_FILE_PATH,SCHEMA_FILE_PATH,PARAMS_FILE_PATH
from src.MLProject.utils.common import read_yaml,create_directories

ImportError: cannot import name 'CONFIG_FILE_PATH' from 'src.MLProject.constants' (unknown location)

In [40]:
print(CONFIG_FILE_PATH)

NameError: name 'CONFIG_FILE_PATH' is not defined

In [39]:
class ConfigurationManager:
 def __init__(
   self,
   config_filepath = CONFIG_FILE_PATH,
   params_filepath = PARAMS_FILE_PATH,
   schema_filepath = SCHEMA_FILE_PATH):
  
    self.config = read_yaml(config_filepath)
    self.params = read_yaml(params_filepath)
    self.schema = read_yaml(schema_filepath)

    create_directories([self.config.artifacts_root])


 def get_data_ingestion_config(self) -> DataIngestionConfig:
   config = self.config.data_ingestion

   create_directories([config.root_dir])

   data_ingestion_config = DataIngestionConfig(
       root_dir=config.root_dir,
       source_URL=config.source_URL,
       local_data_file=config.local_data_file,
       unzip_dir=config.unzip_dir 
   )

   return data_ingestion_config


NameError: name 'CONFIG_FILE_PATH' is not defined